# Import libraries

In [ ]:
import psycopg2
import pandas as pd
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# Initialize Spotify client

In [ ]:
# Set your Spotify API credentials
client_id = '6c2ff081fe494968a2e3a618c678fd62'
client_secret = '99da4179a82f4f999058535d8d9b47dc'

# Initialize the Spotipy client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

# Connect to DB

In [ ]:
# Connect to PostgreSQL database
db_params = {
    "host": "localhost",
    "dbname": "mmh_1",  # Replace with your desired database name
    "user": "postgres",  # Replace with your PostgreSQL username
    "password": "admin",  # Replace with your PostgreSQL password
    "port": "5432" # Replace with your PostgreSQL port
}
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create Views ranking genres for each condition

In [ ]:
# Declare conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

In [ ]:
start = time.process_time()
# Create Views for each condition, ranking genres
for i in range(len(conditions)):
    query = f"""
        CREATE OR REPLACE VIEW {conditions[i]}_music AS(
            SELECT fav_genre, COUNT(*),
                ROW_NUMBER() OVER(ORDER BY COUNT(*) DESC) AS row_n
            FROM survey
            WHERE {conditions[i]} >= 7
            GROUP BY fav_genre
            )
            """
    cur.execute(query,)
    conn.commit()
print(time.process_time() - start)

# Request user input

In [ ]:
# ask user how she feels
user_input = input('How are you feeling today (select a number)?\n  1.Anxious \n  2.Depressed \n  3.Can\'t sleep \n  4.I have OCD \n  5.I feel great!\nEnter a number: ')

# Suggest track(s) according to input

In [ ]:
# look for top choice of genre based on user input (*SQL INJECTION RISK)
if user_input in ['1', '2', '3', '4']:
    x = int(user_input)
    # query fav_genre for anxiety_music
    cur.execute(f"""
                SELECT t.track_name, t.artist, t.album, t.genre, t.preview_url 
                FROM recommended_tracks t
                JOIN {conditions[x-1]}_music a
                ON a.fav_genre = t.genre
                WHERE a.row_n = {random.randint(1,3)};
                """)
    conn.commit()
    result = cur.fetchall()
    #convert result to a dataframe and print    
    df = pd.DataFrame(result, columns=['Song', 'Artist', 'Album', 'Genre', 'Preview URL'])
    print('Have a listen to these songs. They might just cheer you up!\n\n', df)
    
elif user_input == '5':
    # Initialize an empty list to store track data
    all_track_data = []
    # Define seed genres
    seed_genres = ['pagode']
    # Get recommended tracks for each seed genre
    for seed_genre in seed_genres:
        recommended_tracks = sp.recommendations(seed_genres=[seed_genre], limit=1, country="SG")
        for track in recommended_tracks["tracks"]:
            track_data = {
                "track_name": track["name"],
                "artist": ", ".join(artist["name"] for artist in track["artists"]),
                "album": track["album"]["name"],
                "genre": seed_genre,
                "preview_url": track["preview_url"]
                }
            all_track_data.append(track_data)
            # Create a DataFrame from the track data
            df = pd.DataFrame(all_track_data)
            print('Check this out!\n', df)
else:
    print('Hmm, you\'re good, you don\'t need help.')

# Close connection

In [ ]:
# close connection to db
conn.close()